In [1]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
from demoparser.demofile import DemoFile
from matplotlib.patches import ConnectionPatch
import inspect

# Round start

In [2]:
def r_start(event, msg):
    for idx, key in enumerate(event['event'].keys):
        if key.name == 'objective':
            objective = msg.keys[idx].val_string
     
    result.append([objective])

# Round end

In [3]:
def r_end(event, msg):
    for idx, key in enumerate(event['event'].keys):
        if key.name == 'winner':
            winner = msg.keys[idx].val_byte
        elif key.name == 'reason':
            reason = msg.keys[idx].val_byte
        elif key.name == 'legacy':
            legacy = msg.keys[idx].val_byte
        elif key.name == 'message':
            message = msg.keys[idx].val_string
        elif key.name == 'player_count':
            player_count = msg.keys[idx].val_short
     
    result.append([message])

# Match start

In [4]:
def m_start(event, msg):
    result.append("Match start")

# Data processing

In [5]:
def remove_warmup(res_list):
    match_start_idx = len(res_list) - 1 - res_list[::-1].index("Match start")
    new_res_list = res_list[match_start_idx:]
    return new_res_list                

In [6]:
def define_round_num(new_res_list):
    counter = 1
    for i in range(len(new_res_list)):
        if new_res_list[i] in round_end_msgs:
            new_res_list[i] = [{"Round msg": new_res_list[i][0], "round num": counter}]
            counter += 1
    return new_res_list

In [7]:
def remove_trash(new_res_list):
    for el in new_res_list:
        if el == "Match start" or el == ['BOMB TARGET']:
            new_res_list.pop(new_res_list.index(el))
    return new_res_list

In [8]:
def create_lists(new_res_list, att_list, vic_list, r_list):
    round_counter = 1
    
    for el in new_res_list:
        if len(el) == 1:
            round_counter += 1
        else:
            att_list.append(el[0])
            vic_list.append(el[1])
            r_list.append(round_counter)
    return att_list, vic_list, r_list

In [9]:
def split_x_y_z(att_or_vic_list, att_or_vic_x, att_or_vic_y, att_or_vic_z):
    for el in att_or_vic_list:
        att_or_vic_x.append(el['x'])
        att_or_vic_y.append(el['y'])
        att_or_vic_z.append(el['z'])
    
    return att_or_vic_x, att_or_vic_y, att_or_vic_z

In [10]:
def create_df_from_res(att_x, att_y, att_z, vic_x, vic_y, vic_z, rnds):
    data_frame = pd.DataFrame({'attacker_x': att_x,
                       'attacker_y': att_y,
                       'attacker_z': att_z,
                       'victim_x': vic_x,
                       'victim_y': vic_y,
                       'victim_z': vic_z,
                       'round': rnds
                      })
    return data_frame

In [11]:
# points for old dust 2: StartX = -2486; StartY = -1150; EndX = 2127; EndY = 3455; ResX = 1024; ResY = 1024;

def pointx_to_resolutionx(xinput, startX = -2486, endX = 2127, resX = 1024):
    sizeX = endX - startX
    if startX < 0:
        xinput += startX * (-1.0)
    else:
        xinput += startX
    xoutput = float((xinput / abs(sizeX)) * resX);
    return xoutput

def pointy_to_resolutiony(yinput, startY = -1150, endY = 3455, resY = 1024):
    sizeY = endY - startY
    if startY < 0:
        yinput += startY * (-1.0)
    else:
        yinput += startY
    youtput = float((yinput / abs(sizeY)) * resY);
    return resY - youtput - 10

In [12]:
def convert_data(dataframe):
    new_df = pd.DataFrame()
    # Convert the data to radar positions
    new_df['att_map_x'] = dataframe['attacker_x'].apply(pointx_to_resolutionx)
    new_df['att_map_y'] = dataframe['attacker_y'].apply(pointy_to_resolutiony)
    new_df['vic_map_x'] = dataframe['victim_x'].apply(pointx_to_resolutionx)
    new_df['vic_map_y'] = dataframe['victim_y'].apply(pointy_to_resolutiony)
    new_df['round'] = rounds
    
    return new_df

In [13]:
def plot_image(data_frame, list_of_demos):
    fig, ax = plt.subplots(figsize = (20, 20))

    ax.scatter(data_frame.att_map_x, data_frame.att_map_y, alpha = 1, c = 'b')
    ax.scatter(data_frame.vic_map_x, data_frame.vic_map_y, alpha = 1, c = 'r')

    ax.imshow(im)
#     plt.savefig('./images/img_no_lines')
#     for demo in list_of_demos:
#         plt.savefig('./images/{}_no_lines.jpg'.format(demo))
#         plt.close(fig)

In [14]:
def plot_image_with_lines(data_frame, list_of_demos):
    fig, ax = plt.subplots(figsize = (20, 20))

    ax.scatter(data_frame.att_map_x, data_frame.att_map_y, alpha = 1, c = 'b')
    ax.scatter(data_frame.vic_map_x, data_frame.vic_map_y, alpha = 1, c = 'r')

    for i in range(len(data_frame)):
        xyA = data_frame.att_map_x[i], data_frame.att_map_y[i]
        xyB = data_frame.vic_map_x[i], data_frame.vic_map_y[i]

        con = ConnectionPatch(xyA, xyB, coordsA = "data", coordsB = "data",
                              arrowstyle="-", shrinkA=5, shrinkB=5,
                              mutation_scale=20, fc="w")
        ax.add_artist(con)

    ax.imshow(im)
#     plt.savefig('./images/img_lines')
#     for demo in list_of_demos:
#         plt.savefig('./images/{}_lines.jpg'.format(demo))
#         plt.close(fig)

In [15]:
def plot_image_by_rounds(data_frame):
    fig, ax = plt.subplots(data_frame['round'].nunique(), figsize = (150, 150))

    for i in range(data_frame['round'].nunique()):
        ax[i].scatter(data_frame.loc[data_frame['round'] == i + 1].att_map_x, data_frame.loc[data_frame['round'] == i + 1].att_map_y, alpha = 1, c = 'b')
        ax[i].scatter(data_frame.loc[data_frame['round'] == i + 1].vic_map_x, data_frame.loc[data_frame['round'] == i + 1].vic_map_y, alpha = 1, c = 'r')

        ax[i].imshow(im)

In [16]:
def plot_all(data_frame, list_of_demos):
    plot_image(data_frame, list_of_demos)
    #plot_image_with_lines(data_frame, list_of_demos)
    #plot_image_by_rounds(data_frame)

In [17]:
def create_final_res(res_lst):
    new_result = remove_warmup(res_lst)
    res_with_round_num = define_round_num(new_result)
    res_no_trash = remove_trash(res_with_round_num)
    
    return res_no_trash

In [18]:
def all_processes(res):
    final_res = create_final_res(res)
    attackers_list, victims_list, rounds_list = create_lists(final_res, attackers, victims, rounds)
    attackers_x, attackers_y, attackers_z = split_x_y_z(attackers, attacker_x, attacker_y, attacker_z)
    vitctims_x, victims_y, victims_z = split_x_y_z(victims, victim_x, victim_y, victim_z)
    df = create_df_from_res(attacker_x, attacker_y, attacker_z, victim_x, victim_y, victim_z, rounds)
    all_demos_result.append(final_res)
    new_dff = convert_data(df)
    plot_all(new_dff, demos_names)
    
    return all_demos_result

# Demos

In [19]:
demos_list = []
demos_names = []

for file in os.listdir("./data"):
    if file.endswith(".dem"):
        demos_names.append(os.path.splitext(file)[0])
        demos_list.append(os.path.join("./data/", file))

# Player death and run all processes


In [ ]:
all_demos_result = []
im = plt.imread('./maps/de_dust2.png')
round_end_msgs = [['#SFUI_Notice_Bomb_Defused'], ['#SFUI_Notice_CTs_Win'], ['#SFUI_Notice_Target_Saved'], ['#SFUI_Notice_Terrorists_Win'], ['#SFUI_Notice_Target_Bombed']]

def death(event, msg):
    for idx, key in enumerate(event['event'].keys):
        if key.name == 'attacker':
            user_id = msg.keys[idx].val_short
            attacker = d.entities.get_by_user_id(user_id)
        elif key.name == 'userid':
            user_id = msg.keys[idx].val_short
            victim = d.entities.get_by_user_id(user_id)
        elif key.name == 'weapon':
            weapon = msg.keys[idx].val_string
        elif key.name == 'headshot':
            headshot = msg.keys[idx].val_bool 
        
    if attacker and victim:
        result.append([attacker.position, victim.position])

if __name__ == "__main__":
    for demo in demos_list:
        result = []
        attackers, victims, rounds, attacker_x, attacker_y, attacker_z, victim_x, victim_y, victim_z = [], [], [], [], [], [], [], [], []
        data = open(demo, 'rb').read()
        d = DemoFile(data)
        d.add_callback('round_announce_match_start', m_start)
        d.add_callback('round_start', r_start)
        d.add_callback('player_death', death)
        d.add_callback('round_end', r_end)
        d.parse()
        all_demos = all_processes(result)
    print(all_demos)

KeyboardInterrupt: 

Exception ignored in: 'demoparser.demofile.DemoFile.read_new_entity'
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\demoparser\entities\base.py", line 12, in update_prop
    if table in self.props and key in self.props[table]:
KeyboardInterrupt


KeyboardInterrupt: 

Exception ignored in: 'demoparser.demofile.DemoFile.read_new_entity'
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\demoparser\entities\base.py", line 12, in update_prop
    if table in self.props and key in self.props[table]:
KeyboardInterrupt


KeyboardInterrupt: 

Exception ignored in: 'demoparser.demofile.DemoFile.read_new_entity'
Traceback (most recent call last):
  File "D:\Anaconda3\lib\site-packages\demoparser\entities\base.py", line 12, in update_prop
    if table in self.props and key in self.props[table]:
KeyboardInterrupt
